In [1]:
import pandas as pd
import numpy as np

In [21]:
loans_taken_data = pd.read_excel(r'lift_data\Loan Taken Report - Mon Feb 13 2023.xlsx', sheet_name="Loan's Taken")
loans_taken_repayment = pd.read_excel(r'lift_data\Loan Taken Report - Mon Feb 13 2023.xlsx', sheet_name="Loan Repayment Reports")

In [22]:
loans_taken_data.columns

Index(['Respondent ID', 'Gender', 'Age', 'Number of children',
       'Marital Status', 'Country of Residence', 'Citizenship', 'Firm ID',
       'Sector type', 'Date firm established', 'Age of Firm',
       'Number of Owners', 'Owner/s Gender', 'Number of Employees', 'Industry',
       'Location in the Country', 'Loan taken data ID', 'Loan taken name',
       'Loan taken loan tool', 'Loan taken amount taken',
       'Loan taken repayment amount', 'Loan taken remaining amount',
       'Loan taken currency', 'Loan taken date', 'Loan taken loan type',
       'Loan taken interest rate', 'Loan taken payment schedule',
       'Loan taken loan duration', 'Loan taken description',
       'Loan taken surveyor', 'Loan taken account data ID',
       'Loan taken account', 'Loan taken account report ID',
       'Loan taken vendor', 'Loan taken date created',
       'Loan taken last updated', 'Loan taken edit count'],
      dtype='object')

In [23]:
loans_taken_cols = [
    'Respondent ID', 'Loan taken data ID', 'Loan taken name',
    'Loan taken loan tool', 'Loan taken amount taken',
    'Loan taken date', 'Loan taken loan type',
]

In [24]:
loans_taken_filtered = loans_taken_data[loans_taken_cols]

In [25]:
loans_taken_filtered

,Respondent ID,Loan taken data ID,Loan taken name,Loan taken loan tool,Loan taken amount taken,Loan taken date,Loan taken loan type
0,4425,LT00002969,Loan taken 1,Friends,5000.0,2021-09-28,Business
1,4425,LT00003128,Loan taken 2,Supplier,750.0,2021-10-23,Business
2,4425,LT00003330,Loan taken 3,Friends,2750.0,2021-11-16,Business
3,4425,LT00003442,Loan taken 4,Friends,10000.0,2021-12-06,Business
4,4425,LT00003535,Loan taken 5,"Merchandiser, incl. buy-now-pay-later",380.0,2022-01-06,Business
...,...,...,...,...,...,...,...
256,4822,LT00003547,Loan taken 20,Vendor,1250.0,2021-11-09,Business
257,4822,LT00004195,Loan taken 21,Vendor,15000.0,2022-02-12,Business
258,5127,LT00001825,Loan taken 1,Family,322330.0,2020-03-01,Business
259,5127,LT00001826,Loan taken 2,Commercial bank,315000.0,2020-12-28,Business


In [26]:
def check_condition(condition:dict, row: pd.Series) -> bool:
    for key, value in condition.items():
        if type(value) == list:
            found_one = False
            for v in value:
                if v in row[key]:
                    found_one = True
                    break
            if found_one:
                continue
            return False
        elif str(value) in str(row[key]) :
            continue

        return False
    return True

In [27]:
def record_default(row, id_col_name='id'):
    return [row[id_col_name], '', '', '', '']

def record_transaction(cond_map:list[tuple], frame: pd.DataFrame, id_col_name = 'id') -> pd.DataFrame:
    transactions = []
    for row in frame.iterrows():
        recorded = False
        for cond, func in cond_map:
            if check_condition(cond, row[1]):
                transactions.append(func(row[1]))
                recorded = True
                break
        if not recorded:
            transactions.append(record_default(row[1], id_col_name))

    trans_frame = pd.DataFrame(transactions, columns=[id_col_name, 'debit acc', 'debit amt', 'credit acc', 'credit amt'])
    return pd.merge(frame, trans_frame, 'outer', id_col_name)


In [28]:
def record_func_creator(debit_acc, credit_acc, id_col_name, amount_col):
    def func(row: pd.Series):
        return [row[id_col_name], debit_acc, row[amount_col], credit_acc, row[amount_col]]

    return func

In [29]:
AMOUNT_COL = 'Loan taken amount taken'
LOAN_ID = 'Loan taken data ID'
LOAN_TOOL = 'Loan taken loan tool'

CASH = 'cash'
LONG_TERM_LOAN = 'long term loan'
SHORT_TERM_LOAN = 'short term loan'
INFORMAL_LOAN = 'informal loan'
AP = 'AP'
PURCHASES = 'purchases'

condition_func_map = [
    (
        {
            LOAN_TOOL: [
                'Commercial bank', 'Deposit taking microfinance (MDI)'
                'Government bank', 'MFI', 'SACCO/Cooperative'
            ]
        },
        record_func_creator(LONG_TERM_LOAN, CASH, LOAN_ID, AMOUNT_COL)
    ),
    (
        {
            LOAN_TOOL: ['Money lender']
        },
        record_func_creator(SHORT_TERM_LOAN, CASH, LOAN_ID, AMOUNT_COL)
    ),
    (
        {
            LOAN_TOOL: [
                'Friends', 'Other', 'Family',
                'Informal savings group', 'Neighbour', 'Employer',
            ]
        },
        record_func_creator(INFORMAL_LOAN, CASH, LOAN_ID, AMOUNT_COL)
    ),
    (
        {
            LOAN_TOOL: ['Supplier', 'Vendor', 'Merchandiser, incl. buy-now-pay-later']
        },
        record_func_creator(PURCHASES, AP, LOAN_ID, AMOUNT_COL)
    ),
]

In [30]:
loan_taken_transactions = record_transaction(condition_func_map, loans_taken_filtered, LOAN_ID)
loan_taken_transactions.head()

,Respondent ID,Loan taken data ID,Loan taken name,Loan taken loan tool,Loan taken amount taken,Loan taken date,Loan taken loan type,debit acc,debit amt,credit acc,credit amt
0,4425,LT00002969,Loan taken 1,Friends,5000.0,2021-09-28,Business,informal loan,5000.0,cash,5000.0
1,4425,LT00003128,Loan taken 2,Supplier,750.0,2021-10-23,Business,purchases,750.0,AP,750.0
2,4425,LT00003330,Loan taken 3,Friends,2750.0,2021-11-16,Business,informal loan,2750.0,cash,2750.0
3,4425,LT00003442,Loan taken 4,Friends,10000.0,2021-12-06,Business,informal loan,10000.0,cash,10000.0
4,4425,LT00003535,Loan taken 5,"Merchandiser, incl. buy-now-pay-later",380.0,2022-01-06,Business,purchases,380.0,AP,380.0


In [31]:
loans_taken_repayment.columns

Index(['Respondent ID', 'Gender', 'Age', 'Number of children',
       'Marital Status', 'Country of Residence', 'Citizenship', 'Firm ID',
       'Sector type', 'Date firm established', 'Age of Firm',
       'Number of Owners', 'Owner/s Gender', 'Number of Employees', 'Industry',
       'Location in the Country', 'Loan taken repayment report data ID',
       'Loan taken repayment report loan taken data ID',
       'Loan taken repayment report source name',
       'Loan taken repayment report repaid amount',
       'Loan taken repayment report currency',
       'Loan taken repayment report date paid',
       'Loan taken repayment report surveyor',
       'Loan taken repayment report account data ID',
       'Loan taken repayment report account',
       'Loan taken repayment report account report ID',
       'Loan taken repayment report date created',
       'Loan taken repayment report last updated',
       'Loan taken repayment report edit count',
       'Loan taken repayment report des

In [32]:
loans_repay_cols = [
    'Respondent ID', 'Loan taken repayment report data ID',
    'Loan taken repayment report loan taken data ID',
    'Loan taken repayment report repaid amount',
    'Loan taken repayment report date paid',
]
loans_taken_repayment[loans_repay_cols]

,Respondent ID,Loan taken repayment report data ID,Loan taken repayment report loan taken data ID,Loan taken repayment report repaid amount,Loan taken repayment report date paid
0,4425,LP00004064,LT00002969,5000.0,2021-12-06
1,4425,LP00004156,LT00003442,5000.0,2021-12-09
2,4425,LP00004065,LT00003442,5000.0,2021-12-10
3,4425,LP00004066,LT00003442,5000.0,2021-12-10
4,4425,LP00004067,LT00003442,5000.0,2021-12-10
...,...,...,...,...,...
253,5883,LP00002631,LT00001849,9500.0,2021-07-30
254,5883,LP00002630,LT00001849,9500.0,2021-07-30
255,5883,LP00003087,LT00001849,9500.0,2021-08-30
256,5883,LP00003739,LT00001849,19000.0,2021-10-29


In [33]:
loan_repay_filtered = loans_taken_repayment[loans_repay_cols]
loan_repay_filtered.rename(columns={'Loan taken repayment report loan taken data ID': LOAN_ID}, inplace=True)
loan_repay_filtered = pd.merge(loan_repay_filtered, loan_taken_transactions[[LOAN_ID, LOAN_TOOL]], 'left', on=LOAN_ID)
loan_repay_filtered

C:\Users\DIO\AppData\Local\Temp\ipykernel_5956\4224198056.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loan_repay_filtered.rename(columns={'Loan taken repayment report loan taken data ID': LOAN_ID}, inplace=True)


,Respondent ID,Loan taken repayment report data ID,Loan taken data ID,Loan taken repayment report repaid amount,Loan taken repayment report date paid,Loan taken loan tool
0,4425,LP00004064,LT00002969,5000.0,2021-12-06,Friends
1,4425,LP00004156,LT00003442,5000.0,2021-12-09,Friends
2,4425,LP00004065,LT00003442,5000.0,2021-12-10,Friends
3,4425,LP00004066,LT00003442,5000.0,2021-12-10,Friends
4,4425,LP00004067,LT00003442,5000.0,2021-12-10,Friends
...,...,...,...,...,...,...
253,5883,LP00002631,LT00001849,9500.0,2021-07-30,SACCO/Cooperative
254,5883,LP00002630,LT00001849,9500.0,2021-07-30,SACCO/Cooperative
255,5883,LP00003087,LT00001849,9500.0,2021-08-30,SACCO/Cooperative
256,5883,LP00003739,LT00001849,19000.0,2021-10-29,SACCO/Cooperative


In [34]:
REPAY_AMOUNT_COL = 'Loan taken repayment report repaid amount'
LOAN_REPAY_ID = 'Loan taken repayment report data ID'


repay_condition_func_map = [
    (
        {
            LOAN_TOOL: [
                'Commercial bank', 'Deposit taking microfinance (MDI)'
                'Government bank', 'MFI', 'SACCO/Cooperative'
            ]
        },
        record_func_creator(CASH, LONG_TERM_LOAN, LOAN_REPAY_ID, REPAY_AMOUNT_COL)
    ),
    (
        {
            LOAN_TOOL: ['Money lender']
        },
        record_func_creator(CASH, SHORT_TERM_LOAN, LOAN_REPAY_ID, REPAY_AMOUNT_COL)
    ),
    (
        {
            LOAN_TOOL: [
                'Friends', 'Other', 'Family',
                'Informal savings group', 'Neighbour', 'Employer',
            ]
        },
        record_func_creator(CASH, INFORMAL_LOAN, LOAN_REPAY_ID, REPAY_AMOUNT_COL)
    ),
    (
        {
            LOAN_TOOL: ['Supplier', 'Vendor', 'Merchandiser, incl. buy-now-pay-later']
        },
        record_func_creator(AP, CASH, LOAN_REPAY_ID, REPAY_AMOUNT_COL)
    ),
]

In [36]:
loan_repay_transactions = record_transaction(repay_condition_func_map, loan_repay_filtered, LOAN_REPAY_ID)
loan_repay_transactions

,Respondent ID,Loan taken repayment report data ID,Loan taken data ID,Loan taken repayment report repaid amount,Loan taken repayment report date paid,Loan taken loan tool,debit acc,debit amt,credit acc,credit amt
0,4425,LP00004064,LT00002969,5000.0,2021-12-06,Friends,cash,5000.0,informal loan,5000.0
1,4425,LP00004156,LT00003442,5000.0,2021-12-09,Friends,cash,5000.0,informal loan,5000.0
2,4425,LP00004065,LT00003442,5000.0,2021-12-10,Friends,cash,5000.0,informal loan,5000.0
3,4425,LP00004066,LT00003442,5000.0,2021-12-10,Friends,cash,5000.0,informal loan,5000.0
4,4425,LP00004067,LT00003442,5000.0,2021-12-10,Friends,cash,5000.0,informal loan,5000.0
...,...,...,...,...,...,...,...,...,...,...
253,5883,LP00002631,LT00001849,9500.0,2021-07-30,SACCO/Cooperative,cash,9500.0,long term loan,9500.0
254,5883,LP00002630,LT00001849,9500.0,2021-07-30,SACCO/Cooperative,cash,9500.0,long term loan,9500.0
255,5883,LP00003087,LT00001849,9500.0,2021-08-30,SACCO/Cooperative,cash,9500.0,long term loan,9500.0
256,5883,LP00003739,LT00001849,19000.0,2021-10-29,SACCO/Cooperative,cash,19000.0,long term loan,19000.0


In [37]:
loan_repay_transactions.to_csv(r'lift_data_transactions/loan_taken_repayment.csv')

In [38]:
loan_taken_transactions.to_csv(r'lift_data_transactions/loans_taken.csv')